In [ ]:
%cd /content/mdlARC/
from pathlib import Path
import argparse
import importlib
import utils, tinytransformer, train

importlib.reload(utils)  # pick up code changes during iteration
importlib.reload(tinytransformer)
importlib.reload(train)

args = {
    # run config
    "num_workers": 1,
    "device": "cuda",  # 'cuda' | 'mps' | 'cpu'
    # paths - must pass as Path("<path_to_dir>")
    "save_path": Path("runs/tiny.pt"),
    "checkpoint_path": None,  # Path("runs/tiny.pt"),  # or None to start from scratch
    # "data_path": Path("assets/script-tests/grouped-tasks-00d62c1b/challenges.json" ),
    # "data_path": Path("assets/script-tests/grouped-tasks/challenges.json"),
    # "data_path": Path("assets/ARC-1/grouped-tasks/training/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/training/challenges.json"),
    # "data_path": Path("assets/ARC-1/grouped-tasks/combined/challenges.json"),
    # "data_path": Path("assets/ARC-1/grouped-tasks/combined_dihedral_train/challenges.json"),
    # "data_path": Path("assets/ARC-1/grouped-tasks/combined_dihedral_both/challenges.json"),
    # "data_path": Path("assets/ARC-1/grouped-tasks/concept_plus_combined/challenges.json"),
    # "data_path": Path("assets/ARC-1/grouped-tasks/concept_plus_combined_dihedral_train/challenges.json"),
    "data_path": Path(
        "assets/ARC-1/grouped-tasks/concept_plus_combined_dihedral_both/challenges.json"
    ),
    # "data_path": Path("assets/ARC-2/grouped-tasks/combined/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/combined_dihedral_train/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/combined_dihedral_both/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/concept_plus_combined/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/concept_plus_combined_dihedral_train/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/concept_plus_combined_dihedral_both/challenges.json"),
    # hyperparameters
    "epochs": 1,
    "batch_size": 110,
    "val_batch_size": 100,
    "lr": 3e-4,
    "weight_decay": 0.01,
    "grad_clip": 1.0,
    "seed": 42,
    # Visibility toggles
    "log_train_strings": False,
    "log_train_limit": 10,
    "log_inference_prompt": False,
}
cfg = argparse.Namespace(**args)
try:
    reuse_dataset = dataset
except NameError:
    reuse_dataset = None
model, dataset, dataloader, device, data_path = train.build_model_and_data(
    cfg, reuse_dataset=reuse_dataset
)

In [ ]:
# Training only
train.train_model(
    cfg,
    model=model,
    dataloader=dataloader,
    dataset=dataset,
    device=device,
    data_path=data_path,
)

In [ ]:
# inference + visualisation check
import inference
from utils import plot_grids, split_grids_from_tokens, tokens_to_string

importlib.reload(utils)
importlib.reload(inference)

task_ids_list = ["00d62c1b", "e0fb7511", "00576224", "3aa6fb7a"]  # always pass as list
# task_ids_list = ["da515329", "e12f9a14", "88e364bc", "a251c730", "9385bd28", "409aa875", "abc82100", "a6f40cea", "d59b0160", "62593bfd", "9aaea919", "b9e38dc0", "64efde09", "2b83f449", "eee78d87", "195c6913", "db695cfb", "67e490f4", "6e4f6532", "e87109e9", "581f7754", "a25697e4", "97d7923e", "58f5dbd5", "1ae2feb7", "16de56c4", "b5ca7ac4", "291dc1e1", "446ef5d2", "dd6b8c4b", "38007db0", "28a6681f", "8b7bacbf", "edb79dae", "31f7f899", "4c3d4a41", "de809cff", "898e7135", "fc7cae8d", "6e453dd6", "c4d067a0", "8698868d", "8b9c3697", "89565ca0", "53fb4810", "e3721c99", "e8686506", "35ab12c3", "20270e3b", "88bcf3b4", "7491f3cf", "7b80bb43", "dfadab01", "a47bf94d", "3dc255db", "2c181942", "6ffbe589", "faa9f03d", "332f06d7", "2ba387bc", "800d221b", "5dbc8537", "b99e7126", "71e489b6", "3a25b0d8", "5545f144", "7c66cb00", "cb2d8a2c", "135a2760", "78332cb0", "221dfab4", "271d71e2", "4c416de3", "4e34c42c", "20a9e565", "f560132c", "3e6067c3", "7666fa5d", "dbff022c", "db0c5428", "45a5af55", "7b0280bc", "0934a4d8", "80a900e0", "4a21e3da", "f931b4a8", "142ca369", "7ed72f31", "aa4ec2a5", "21897d95", "2d0172a1", "7b3084d4", "b10624e5", "58490d8a", "269e22fb", "1818057f", "b6f77b65", "16b78196", "136b0064", "65b59efc", "8e5c0c38", "981571dc", "4c7dc4dd", "8f3a5a89", "36a08778", "b0039139", "cbebaa4b", "247ef758", "a395ee82", "c7f57c3e", "13e47133", "8f215267", "7b5033c1", "e376de54", "a32d8b75", "bf45cf4b", "5961cc34", "d8e07eb2", "d35bdbdc", "9bbf930d"]
# task_ids_list = ["00576224", "009d5c81", "00dbd492", "03560426", "05a7bcf2", "0607ce86", "0692e18c", "070dd51e", "08573cc6", "0934a4d8", "09c534e7", "0a1d4ef5", "0a2355a6", "0b17323b", "0bb8deee", "0becf7df", "0c786b71", "0c9aba6e", "0d87d2a6", "0e671a1a", "0f63c0b9", "103eff5b", "11e1fe23", "12422b43", "12997ef3", "12eac192", "136b0064", "13713586", "137f0df0", "140c817e", "14754a24", "15113be4", "15663ba9", "15696249", "16b78196", "17b80ad2", "17cae0c1", "18419cfa", "184a9768", "195ba7dc", "1990f7a8", "19bb5feb", "1a2e2828", "1a6449f1", "1acc24af", "1c02dbbe", "1c0d0a4b", "1c56ad9f", "1d0a4b61", "1d398264", "1da012fc", "1e81d6f9", "1e97544e", "2037f2c7", "2072aba6", "20818e16", "20981f0e", "212895b5", "21f83797", "22a4bbc2", "25094a63", "2546ccf6", "256b0a75", "2685904e", "2697da3f", "2753e76c", "27a77e38", "27f8ce4f", "281123b4", "292dd178", "29700607", "2a5f8217", "2b01abd0", "2c0b0aff", "2c737e39", "2f0c5170", "310f3251", "3194b014", "319f2597", "31adaf00", "31d5ba1a", "32e9702f", "332efdb3", "3391f8c0", "33b52de3", "3490cc26", "34b99a2b", "351d6448", "358ba94e", "37d3e8b2", "3979b1a8", "3a301edc", "3b4c2228", "3d31c5b3", "3ed85e70", "3ee1011a", "3f23242b", "40f6cd08", "414297c0", "423a55dc", "42918530", "42a15761", "4364c1c4", "456873bc", "45737921", "45bbe264", "477d2879", "47996f11", "48131b3c", "4852f2fa", "48f8583b", "4aab4007", "4acc7107", "4b6b68e5", "4c177718", "4cd1b7b2", "4e45f183", "4e469f39", "4f537728", "4ff4c9da", "505fff84", "506d28a5", "50a16a69", "50aad11f", "50f325b5", "516b51b7", "5207a7b5", "5289ad53", "52fd389e", "54db823b", "55059096", "551d5bf1", "55783887", "575b1a71", "5783df64", "5833af48", "58743b76", "58e15b12", "59341089", "5a5a2103", "5af49b42", "5b526a93", "5b692c0f", "5b6cbef5", "5d2a5c43", "5ffb2104", "604001fa", "60a26a3e", "60c09cac", "626c0bcc", "62ab2642", "62b74c02", "639f5a19", "642248e4", "642d658d", "64a7c07e", "66e6c45b", "66f2d22f", "67636eac", "67b4a34d", "67c52801", "68b67ca3", "692cd3b6", "695367ec", "696d4842", "69889d6e", "6a11f6da", "6ad5bdfd", "6df30ad6", "6ea4a07e", "6f473927", "7039b2d7", "705a3229", "712bf12e", "72207abc", "72a961c9", "73182012", "73c3b0d8", "73ccf9c2", "759f3fd3", "762cd429", "770cc55f", "782b5218", "79369cc6", "7953d61e", "79fb03f4", "7bb29440", "7c8af763", "7c9b52a0", "7d18a6fb", "7d1f7ee8", "7d419a02", "7e02026e", "7ee1c6ea", "817e6c09", "81c0276b", "833dafe3", "845d6e51", "84db8fc4", "84f2aca1", "8597cfd7", "85b81ff1", "85fa5666", "8719f442", "88207623", "891232d6", "896d5239", "8a371977", "8b28cd80", "8ba14f53", "8cb8642d", "8dae5dfc", "8e2edd66", "8ee62060", "8fbca751", "90347967", "903d1b4a", "9110e3c5", "917bccba", "929ab4e9", "92e50de0", "9356391f", "93b4f4b3", "93c31fbe", "94133066", "94414823", "94be5b80", "95a58926", "963f59bc", "96a8c0cd", "97239e3d", "9772c176", "981571dc", "992798f6", "99306f82", "9a4bb226", "9b2a60aa", "9b365c51", "9b4c17c4", "9bebae7a", "9c1e755f", "9c56f360", "9caba7c3", "9ddd00f0", "9def23fe", "9f27f097", "a04b2602", "a096bf4d", "a3f84088", "a406ac07", "a57f2f04", "a59b95c0", "a680ac02", "a8610ef7", "a934301b", "aa18de87", "aa300dc3", "aa4ec2a5", "aab50785", "ac0c5833", "ac2e8ecf", "ac3e2b04", "ac605cbb", "ad7e01d0", "ae58858e", "aee291af", "af22c60d", "af24b4cc", "b0722778", "b0f4d537", "b15fca0b", "b1fc8b8e", "b20f7c8b", "b457fec5", "b4a43f3b", "b7999b51", "b7cb93ac", "b7f8a4d8", "b7fb29bc", "b942fd60", "b9630600", "ba9d41b8", "baf41dbf", "bb52a14b", "bbb1b8b6", "bc4146bd", "bcb3040b", "bd14c3bf", "be03b35f", "bf32578f", "bf699163", "bf89d739", "c074846d", "c1990cce", "c3202e5a", "c35c1b4c", "c48954c1", "c62e2108", "c64f1187", "c658a4bd", "c663677b", "c6e1b8da", "c7d4e6ad", "c87289bb", "c8b7cc0f", "c92b942c", "c97c0139", "ca8de6ea", "ca8f78db", "cad67732", "cb227835", "ccd554ac", "cd3c21df", "ce039d91", "ce8d95cc", "cf133acc", "cfb2ce5a", "d017b73f", "d19f7514", "d282b262", "d2acf2cb", "d304284e", "d37a1ef5", "d47aa2ff", "d492a647", "d4b1c2b1", "d4c90558", "d56f2372", "d5c634a2", "d931c21c", "d94c3b52", "da2b0fe3", "da515329", "dc2aa30b", "dc2e9a9d", "dd2401ed", "de493100", "df8cc377", "e0fb7511", "e133d23d", "e1baa8a4", "e1d2900e", "e2092e0c", "e21a174a", "e345f17b", "e4075551", "e41c6fd3", "e57337a4", "e5790162", "e5c44e8f", "e619ca6e", "e633a9e5", "e66aafb8", "e681b708", "e69241bd", "e6de6e8f", "e74e1818", "e760a62e", "e7639916", "e78887d1", "e7a25a18", "e7b06bea", "e7dd8335", "e872b94a", "e88171ec", "e95e3d8e", "e99362f0", "e9ac8c9e", "e9b4f6fc", "e9bb6954", "e9c9d9a1", "ea959feb", "ea9794b1", "ecaa0ec1", "ed74f2f2", "ed98d772", "ef26cbf6", "f0afb749", "f0df5ff0", "f21745ec", "f3b10344", "f3cdc58f", "f3e62deb", "f4081712", "f45f5ca7", "f5aa3634", "f5c89df1", "f823c43c", "f83cb3f6", "f8be4b64", "f9a67cb5", "f9d67f8b", "fafd9572", "fb791726", "fc754716", "fd096ab6", "fd4b2b02", "fe9372f3", "fea12743", "ff72ca3e"]
# selected_split = "test"
selected_split = "train"
pair_idx = 0
visualise = True

results = inference.run_batched_inference(
    model=model,
    dataset=dataset,
    task_ids=task_ids_list,
    device=device,
    split=selected_split,
    pair_index=pair_idx,
    include_targets=True,
)

if not results:
    print("No inference results were produced.")
for res in results:
    print(f"\nTask {res['task_id']} pair {res['pair_index']} ({selected_split})")
    print("Prompt tokens:", tokens_to_string(res["prompt_tokens"]))
    print("Generated output tokens:", tokens_to_string(res["output_tokens"]))
    if res.get("target_output_tokens"):
        print("Target output tokens:", tokens_to_string(res["target_output_tokens"]))
    print("Predicted grid:")
    for row in res["output_grid"]:
        print(row)
    if res.get("target_grid"):
        print("Target grid:")
        for row in res["target_grid"]:
            print(row)
    if visualise:
        prompt_grids = split_grids_from_tokens(res["prompt_tokens"])
        input_grid = prompt_grids[0] if prompt_grids else []
        to_plot = [input_grid, res["output_grid"]]
        if res.get("target_grid"):
            to_plot.append(res["target_grid"])
        try:
            plot_grids(
                to_plot,
                title=f"{res['task_id']} pair {res['pair_index']} ({selected_split})",
            )
        except ValueError:
            print(
                f"skipping visualisation for {res['task_id']} pair {res['pair_index']} ({selected_split})"
            )


In [ ]:
# Full dataset evaluation
import inference

importlib.reload(inference)

EVAL_BATCH_SIZE = 1000
splits = ["test"]

evaluation = inference.evaluate_model_on_dataset(
    model=model,
    dataset=dataset,
    device=device,
    batch_size=EVAL_BATCH_SIZE,
    log_prompts=args["log_inference_prompt"],
    splits=splits,
)

for split in splits:
    summary = evaluation.get(split, {}).get("summary", {})
    total = summary.get("total_sequences", 0)
    shape_ok = summary.get("num_shape_correct", 0)
    avg_pixel_acc = summary.get("avg_pixel_accuracy", 0.0)
    fully_correct = summary.get("num_fully_correct", 0)

    print(f"\nSplit: {split}")
    print(f"  sequences evaluated: {total}")
    print(f"  correct output grid shapes: {shape_ok} / {total}")
    if shape_ok > 0:
        print(f"  avg pixel accuracy (shape-correct only): {avg_pixel_acc:.4f}")
    else:
        print("  avg pixel accuracy (shape-correct only): n/a")
    print(f"  fully correct output grids: {fully_correct} / {total}")

    if split == "test":
        correct_outputs = summary.get("fully_correct_results", [])
        print("  fully correct test outputs (task_id, pair_index, grid):")
        if not correct_outputs:
            print("    (none)")
        for res in correct_outputs:
            grid = res.get("output_grid", [])
            print(f"    - {res.get('task_id')} pair {res.get('pair_index')}: {grid}")


In [ ]:
# AAIVR voting on augmented test predictions
import utils

test_results = evaluation.get("test", {}).get("results", [])
aaivr_results = utils.run_aaivr_on_results(test_results)

print("\nAAIVR selections (pass@2) for test split:")
if not aaivr_results:
    print("  no test results available for AAIVR voting")

for res in aaivr_results:
    print(f"- task {res.task_id} original_pair {res.original_pair_index}")
    print(
        f"  generated={res.num_generated}, valid_after_filter={res.num_valid}, "
        f"dropped_non_rectangular={res.discarded_non_rectangular}, "
        f"dropped_input_copies={res.discarded_input_copies}"
    )
    if not res.ranked_candidates:
        print("  no valid candidates after filtering")
        continue
    for idx, cand in enumerate(res.ranked_candidates[:2]):
        grid = cand.get("grid", [])
        count = cand.get("count", 0)
        print(f"  option {idx + 1} (count={count}): {grid}")

summary = utils.summarize_aaivr_pass_at_k(aaivr_results)
evaluated = summary.get("evaluated", 0)
hits = summary.get("hits", 0)
print("AAIVR pass@2 with targets:", f"{hits} / {evaluated} original test pairs")


In [ ]:
%cd /content/mdlARC/
import json

formatted_eval = inference.group_eval_sequences_by_task(evaluation)
cfg.checkpoint_path = Path("runs/tiny.pt")
json_path = cfg.checkpoint_path.with_suffix(".json")
with open(json_path, "w") as f:
    json.dump(formatted_eval, f, indent=2)
print(f"Saved grouped task sequences to: {json_path}")

# Large scale training run (periodic checkpointing and evaluation)

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# config
%cd /content/mdlARC/
from pathlib import Path
import argparse
import importlib
import utils, train

importlib.reload(utils)  # pick up code changes during iteration
importlib.reload(train)

args = {
    # run config
    "num_workers": 0,
    "device": "cuda",  # 'cuda' | 'mps' | 'cpu'
    # paths - must pass as Path("<path_to_dir>")
    "train_log_file": Path("runs/train_log.txt"),
    # "data_path": Path("assets/script-tests/grouped-tasks-00d62c1b/challenges.json" ),
    # "data_path": Path("assets/script-tests/grouped-tasks/challenges.json"),
    # "data_path": Path("assets/ARC-1/grouped-tasks/training/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/training/challenges.json"),
    # "data_path": Path("assets/ARC-1/grouped-tasks/combined/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/combined/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/combined_dihedral_train/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/combined_dihedral_both/challenges.json"),
    # "data_path": Path("assets/ARC-2/grouped-tasks/concept_plus_combined/challenges.json"),
    "data_path": Path(
        "assets/ARC-2/grouped-tasks/concept_plus_combined_dihedral_train/challenges.json"
    ),
    # "data_path": Path("assets/ARC-2/grouped-tasks/concept_plus_combined_dihedral_both/challenges.json"),
    # hyperparameters
    "batch_size": 60,
    "val_batch_size": 500,
    "lr": 3e-4,
    "weight_decay": 0.01,
    "grad_clip": 1.0,
    "seed": 42,
    # Visibility toggles
    "log_train_strings": False,
    "log_train_limit": 10,
    "log_inference_prompt": False,
}
cfg = argparse.Namespace(**args)


In [ ]:
# Training with periodic checkpointing

cfg.epochs = 200
n_cycles = 3

# Make base 0 if training from scratch. Otherwise, last checkpoint number
cfg.base = 0

for i in range(n_cycles):
    cfg.save_path = Path(f"runs/tiny-{(i + 1) * cfg.epochs + cfg.base}.pt")

    if cfg.base == 0 and i == 0:
        cfg.checkpoint_path = None
        reuse_dataset = None
    else:
        cfg.checkpoint_path = Path(f"runs/tiny-{i * cfg.epochs + cfg.base}.pt")
        try:
            reuse_dataset = dataset
        except NameError:
            reuse_dataset = None  # if `dataset` was deleted from memory

    model, dataset, dataloader, device, data_path = train.build_model_and_data(
        cfg, reuse_dataset=reuse_dataset
    )
    train.train_model(
        cfg,
        model=model,
        dataloader=dataloader,
        dataset=dataset,
        device=device,
        data_path=data_path,
    )

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# evaluate all the checkpoints
import json
import inference

importlib.reload(inference)

EVAL_BATCH_SIZE = 500
eval_log_path = Path("runs/eval_log.txt")

# only for debugging. keep commented out
# cfg.base = 0
# cfg.epochs = 200
# n_cycles = 1

try:
    reuse_dataset = dataset
except NameError:
    reuse_dataset = None

for i in range(n_cycles):
    cfg.checkpoint_path = Path(f"runs/tiny-{(i + 1) * cfg.epochs + cfg.base}.pt")
    model, dataset, dataloader, device, data_path = train.build_model_and_data(
        cfg, reuse_dataset=reuse_dataset
    )
    evaluation = inference.evaluate_model_on_dataset(
        model=model,
        dataset=dataset,
        device=device,
        batch_size=EVAL_BATCH_SIZE,
        log_prompts=cfg.log_inference_prompt,
    )

    formatted_eval = inference.group_eval_sequences_by_task(evaluation)

    json_path = cfg.checkpoint_path.with_suffix(".json")
    with open(json_path, "w") as f:
        json.dump(formatted_eval, f, indent=2)
    print(f"Saved grouped task sequences to: {json_path}")

    with open(eval_log_path, "a") as f:
        f.write(f"--- Checkpoint: {(i + 1) * cfg.epochs + cfg.base} ---\n")
        for split_name, data in evaluation.items():
            summary = data["summary"]
            log_line = (
                f"Split: {split_name:<6} | "
                f"Total: {summary['total_sequences']:<4} | "
                f"Shape Correct: {summary['num_shape_correct']:<4} | "
                f"Fully Correct: {summary['num_fully_correct']:<4} | "
                f"Pixel Acc: {summary['avg_pixel_accuracy']:.4f}"
            )
            print(log_line)
            f.write(log_line + "\n")
            if split_name == "test":
                correct_outputs = summary.get("fully_correct_results", [])
                log_line = "  fully correct test outputs (task_id, pair_index, grid):"
                if not correct_outputs:
                    log_line = log_line + "\n    (none)"
                for res in correct_outputs:
                    grid = res.get("output_grid", [])
                    log_line = log_line + (
                        f"\n    - {res.get('task_id')} pair {res.get('pair_index')}: {grid}"
                    )
                print(log_line)
                f.write(log_line + "\n")

        f.write("\n\n")
        print("\n")


In [ ]:
import shutil
from datetime import datetime

timestamp = datetime.now().strftime("%d%m%y-%H%M%S")

src_dir = "/content/mdlARC/runs"  # folder to back up
zip_base = "/content/mdlARC/runs_compressed"  # zip will become this + '.zip'
dst_zip = f"/content/drive/MyDrive/run_{timestamp}.zip"

# Create zip in /content
shutil.make_archive(zip_base, "zip", src_dir)

# Copy zip into Drive
shutil.copy2(zip_base + ".zip", dst_zip)